In [ ]:
import sys
sys.path.append("..")


import datetime as dt
from ipywidgets import interact
import importlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import statsmodels.api as sm
import tqdm
import vectorbtpro as vbt

from lib import utils, indicators, signal

In [ ]:
symbol = 'BTCUSDT'
freq = '5min'

start = dt.date(2024,1,1)
end = dt.date(2024,3,30)

df = pd.read_parquet(f"../data/binance/{symbol}.pq").loc[start:end]
df = df.resample(freq).agg({'timestamp':'first', 'close_time':'last', 'open':'first', 'high':'max', 'low':'min', 'close':'last', 'volume':'sum', 'quoteVolume':'sum', 'nTrades':'sum', 'upVolume':'sum', 'upQuoteVolume':'sum'})
indicators.Standard(df)

In [ ]:
importlib.reload(signal)
class MACross(signal.Signal):
    def generate_signals(self, df, params):
        df['ma1'] = indicators.SuperSmoother(df['hlcc4'], params['ma1'])
        df['ma2'] = indicators.GaussianFilter(df['hlcc4'], params['ma2'], 2)
        return {
            'le': utils.crossover(df['ma1'], df['ma2']),
            'se': utils.crossover(df['ma2'], df['ma1']),
            'lx': None,
            'sx': None
            }

    def optimize_signal(self, df):
        return {}

In [ ]:
p = {
    'ma1': 21,
    'ma2': 200,
}
strat = MACross()
signals = strat.generate_signals(df, p)

In [ ]:
strat.plot_signal_sltp_stop_natr(df, p)

In [ ]:
strat.plot_signal_td_stop(df, p)